In [30]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import mplfinance as mpl
import os


#install symbols nasdaq100の銘柄コード

# df = pd.read_csv('./nasdaq100.csv')

# sign = []
# for symbol in df['symbol']:
#     sign.append(symbol)


sign =['ATVI', 'AMD', 'ALXN', 'ALGN', 'GOOGL', 'GOOG', 'AMZN', 'AEP', 'AMGN', 'ADI'
        , 'ANSYS', 'ANSS', 'AAPL', 'AMAT', 'ASML', 'TEAM', 'ADSK', 'ADP', 'BIDU', 'BIIB', 'BKNG'
        , 'AVGO', 'CDNS', 'CDW', 'CDW', 'CERN', 'CHTR', 'CHKP', 'CTAS', 'CSCO', 'CTSH', 'CMCSA'
        , 'CPRT', 'COST', 'CSX', 'DXCM', 'DOCU', 'DLTR', 'EBAY', 'EA', 'EXC', 'FB', 'FAST'
        , 'FISV', 'FOXA', 'FOX', 'GILD', 'IDXX', 'ILMN', 'INCY', 'INTC', 'INTU', 'ISRG', 'JD'
        , 'KDP', 'KLAC', 'KHC', 'LRCX', 'LULU', 'MAR', 'MRVL', 'MTCH', 'MXIM', 'MELI', 'MCHP', 'MU'
        , 'MSFT', 'MRNA', 'MDLZ', 'MNST', 'NTES', 'NFLX', 'NVDA', 'NXPI', 'ORLY', 'OKTA', 'PCAR', 'PAYX'
        , 'PYPL', 'PTON', 'PEP', 'PDD', 'QUALCOMM', 'QCOM', 'REGN', 'ROST', 'SGEN', 'SIRI', 'SWKS', 'SPLK'
        , 'SBUX', 'SNPS', 'TMUS', 'TSLA', 'TXN', 'TCOM', 'VRSN', 'VRSK', 'VRTX', 'WBA', 'WDAY', 'XEL', 'XLNX'
        , 'ZM']

totalresult = []

for i in sign:
    stock_code = i 
    start = '2020-01-01'
    end = '2021-02-02'
    df = yf.download(stock_code, start, end, interval='1d') 
    
        
    
    print(i)
    term=81
    # chek the highest price in the past {term} times
    df['Highest'+str(term)] = df.iloc[:, 4].rolling(window=term).max()
    # chek the highest price in the past {term} times
    df['Lowest'+str(term)] = df.iloc[:, 4].rolling(window=term).min()
    
    # chek the highest price in the past {term} times 手仕舞い用
    termN = 23
    df['Lowest'+str(termN)] = df.iloc[:, 4].rolling(window=termN).min()
    
        
    #calucurate sma
     # calculation SMA
    short_sma = 20
    long_sma = 50
    verylong_sma = 250
   

    """
    メインループ
    基本戦略
    turtles strategy only
    過去longTerm間の調整済み終値の最高値をその日の調整済み終値が更新したら購入
    損切り率を上げていく
    最終日にまだ保持していた場合捌く
    """
    # 資産、購入したときの資産
    assets = [1]
    buy_asset = 1
    # 買いポジションは０にする
    buy_position = False
   
    # 損切り率
    stopLoss = 0.94
 
    
    #買い日と売り日
    buy_days = []
    sell_days = []
    
    #holding period list
    holding_period_list = []



    counter=0
    percentChange = []
    for i in range(1, len(df)):
        Highest = df['Highest'+str(term)][i-1]
        Lowest = df['Lowest'+str(term)][i-1]
        LowestN = df['Lowest'+str(termN)][i-1]
        high_price = df['High'][i]
        low_price = df['Low'][i]
        close = df['Adj Close'][i]
        open_price = df['Open'][i]
        last_close = df['Close'][i-1]
        last_high = df['High'][i-1]
        


        if buy_position == False:
            assets.append(assets[i-1])
        else:
            assets.append(buy_asset*(close / buy_price))

        # avoid NaN data 買いトレンド
        if np.isnan( Highest) == False:
            if close > Highest and buy_position == False :
                buy_position = True
                buy_price = close
                buy_stopLine = close * stopLoss
                buy_asset = assets[i]
                buyday = pd.to_datetime(df.index[i])
                buy_days.append(buyday)
                current_stop_level = 0
                buypoint = i
                print('buy in {} at {}'.format(buyday,buy_price))
                
        if buy_position == True and close > buy_price * 1.05 and current_stop_level == 0:
            buy_stopLine = buy_price
            current_stop_level = 1
            print('up stopline1 in {} at {}'.format(df.index[i],buy_stopLine))
            
        if buy_position == True and close > buy_price * 1.1 and current_stop_level == 1:
            buy_stopLine = buy_price * 1.05
            current_stop_level = 2
            print('up stopline2 in {} at {}'.format(df.index[i],buy_stopLine))
           
            
        if buy_position == True and close > buy_price * 1.2 and current_stop_level == 2:
            buy_stopLine = buy_price * 1.1
            current_stop_level = 3
            print('up stopline3 in {} at {}'.format(df.index[i],buy_stopLine))
            
        if buy_position == True and close > buy_price * 1.3 and current_stop_level == 3:
            buy_stopLine = close * 0.85
            current_stop_level = 4
            print('up stopline4 in {} at {}'.format(df.index[i],buy_stopLine))
            
        if buy_position == True and close > buy_price * 1.5 and current_stop_level == 4:
            buy_stopLine = close * 0.8
            current_stop_level = 5  
            print('up stopline5 in {} at {}'.format(df.index[i],buy_stopLine))
            
            
        if buy_position == True and close > Highest and current_stop_level == 5:
            buy_stopLine = close * 0.8
            current_stop_level = 5  
            print('up stopline6 in {} at {}'.format(df.index[i],buy_stopLine))
            

         # 損切り or Profit taking
        if buy_position == True and open_price < buy_stopLine and i != buypoint:
            buy_position = False 
            percent = (open_price/buy_price - 1) * 100
            percentChange.append(percent)
            sellday = pd.to_datetime(df.index[i])
            sell_days.append(sellday)
            holding_period = sellday - buyday
            print('sell in {} at {} | profit_one is {}'.format(sellday,open_price,percent))
            
        elif buy_position == True and close < buy_stopLine and i != buypoint:
            buy_position = False 
            percent = (buy_stopLine/buy_price - 1) * 100
            percentChange.append(percent)
            sellday = pd.to_datetime(df.index[i])
            sell_days.append(sellday)
            holding_period = sellday - buyday
            holding_period_list.append(holding_period)
            print('sell in {} at {} | profit_two is {}'.format(sellday,buy_stopLine,percent))

        #  最終日にまだ持ってたら利確する
        if i == (len(df)-1) and buy_position == True:
            buy_position = False
            percent = (close / buy_price - 1) * 100
            percentChange.append(percent)
            sellday = pd.to_datetime(df.index[i])
            sell_days.append(sellday)
            holding_period = sellday - buyday
            holding_period_list.append(holding_period)
            print('Date:'+str(i),'Last Day Buy at the price {}'.format(close))

        counter += 1


    #print(percentChange)

    # statistic
#     ------------------------------------------
    gains = 0
    losses = 0
    numlosses = 0
    numgains = 0
    total_return = 1


    for i in percentChange:
        if i > 0:
            numgains += 1
            gains += i
        else:
            numlosses += 1
            losses += i
        total_return = total_return * ((i / 100) + 1)
        
        
    total_return = round((total_return - 1)*100, 2)
    if numgains > 0:
        ave_gain = gains / numgains
#         max_return = max(percentChange)
    else:
        ave_gain = 0
        max_return = 'unknown'
    
    if numlosses > 0:
        ave_loss = losses / numlosses
#         max_loss = min(percentChange)
        risk_reward_retio = - ave_gain / ave_loss
    else:
        ave_loss = 0
        max_loss = 'unknown'
        risk_reward_retio = 'inf'
    
    if numgains > 0 or numlosses > 0:
        batting_retio = numgains / (numgains + numlosses)
    else:
        batting_retio = 0
    
    trades = numgains + numlosses
    ave_total = ave_gain * batting_retio + ave_loss * (1 - batting_retio)
    
    #average holding period
    if len(holding_period_list) != 0:   
        average_holding_period = sum(holding_period_list)/len(holding_period_list)
    

    results = [numgains+numlosses, round(total_return,2), round(ave_gain,2), round(ave_loss,2)
               ,batting_retio ,ave_total ,average_holding_period]
    
    if not results is None:
         totalresult.append(results)
            

            
    #買いポイント描写
#     buy_signals = []
#     sell_signals = []
#     for i in df.index:
#         if i in buy_days:
#             buy_signals.append(df['Adj Close'][i])
#         else:
#             buy_signals.append(np.nan)
#         if i in sell_days:
#             sell_signals.append(df['Adj Close'][i])
#         else:
#             sell_signals.append(np.nan)
#     adp = [
#         mpl.make_addplot(buy_signals, type='scatter', markersize=10000, marker='|', color="b"),
#         mpl.make_addplot(sell_signals, type='scatter', markersize=10000, marker='|', color="r")]
#     fig = mpl.plot(df, type='candle',figsize=(30, 15),mav=(short_sma, long_sma, verylong_sma),
#                addplot=adp, style='yahoo', volume=True,  title=str(stock_code))



# print(totalresult)

columns = ['trades', 'Total return', 'Average Gain', 'Average Loss', '勝率' ,'Average gain and profit','average holding period']
df = pd.DataFrame(totalresult, columns=columns, index = sign)
df.to_csv('./turtle_result.csv')

print(df)


[*********************100%***********************]  1 of 1 completed
ATVI
buy in 2020-05-05 00:00:00 at 68.52999877929688
up stopline1 in 2020-05-06 00:00:00 at 68.52999877929688
up stopline2 in 2020-05-11 00:00:00 at 71.95649871826173
sell in 2020-05-26 00:00:00 at 71.95649871826173 | profit_two is 5.000000000000004
buy in 2020-06-17 00:00:00 at 75.52999877929688
up stopline1 in 2020-07-09 00:00:00 at 75.52999877929688
up stopline2 in 2020-08-03 00:00:00 at 79.30649871826172
sell in 2020-08-11 00:00:00 at 79.30649871826172 | profit_two is 5.000000000000004
buy in 2020-12-14 00:00:00 at 85.94000244140625
up stopline1 in 2020-12-18 00:00:00 at 85.94000244140625
up stopline2 in 2021-01-20 00:00:00 at 90.23700256347657
sell in 2021-01-27 00:00:00 at 90.23700256347657 | profit_two is 5.000000000000004


TypeError: unsupported operand type(s) for +: 'int' and 'Timedelta'